# Development of initial conditions for wave height & period
## Notebook #1  

*PYTHON*

#### Overview: Use Hurdat2 dataset to select storms within area of interest

1. Copy most recent HURDAT2 Data to text file & save in root directory
 [(*HURDAT2 data description*)](http://www.nhc.noaa.gov/data/hurdat/hurdat2-format-atlantic.pdf)
- Assign directories & write a hurdat file suitable for import into GIS
- Geoprocessing Step: 
        a. In GIS, create shapefile from text file (using lat_d & lon_d for x,y)
        b. Select Points within a buffered region (a reasonable distance fom the study area)
            i. Use a buffer defining the area of interest
            ii. Use a second (slighly bigger buffer) to ensure enough points are captured to compute velocities
        c. Project points to State Plane (use feet --or-- carefully check all calculations in these 
            notebooks & change to meters)
        d. Add fields for x,y (e.g. lat_state_plane) and calculate geometry in the reference coordinates 
            for unit consistency
- Read in Post-Geoprocessed data & format file for further processing
- For each storm, compute forward speed based on hurdat points in area of interest
- Select data of interest & Merge computed velocities into a new dataframe
- Code in data from [Reference Guidance](http://www.nws.noaa.gov/oh/hdsc/Technical_memoranda/TM23.pdf) (Location specific) to augment statistical dataset
- Select storm categories to include (e.g. Hurricanes) & combine Central Pressure values into a single column
- Calculate Radius to Mad Wind [(Equation)](https://rdrr.io/cran/stormwindmodel/man/will7a.html) for all storms & write the final dataset to file




### 1. Copy most recent [HURDAT2 Data](http://www.aoml.noaa.gov/hrd/hurdat/hurdat2-1851-2016-apr2017.txt) to text file & save in root directory

### 2. Assign directories & write a hurdat file suitable for import into GIS

In [1]:
# import libraries
import os
import warnings
import pandas as pd
import numpy as np
%matplotlib inline
warnings.filterwarnings('ignore')

In [2]:
# Assign Directories
root_dir = '/Users/slawler/Repos/PuertoRico-WaveConditions/data'
hurdat_file = 'hurdat2.txt' # input file
hurdat_file_reformatted = 'hurdat_reform.csv'# output file (created in step 2)
hurdat_post_gis = 'pr_storms_state_plane.txt' # input file (created in step 3)
processed_data = 'Storms_for_WaveCalcs.tsv' # output file (created in step 9)

hurdat = os.path.join(root_dir, hurdat_file)
hurdat_reformat = os.path.join(root_dir, hurdat_file_reformatted)

In [3]:
# Column header data
ColHead = 'DateID,Name,Date,HourUTC,RecID,Status,Lat,Long,WindSpd,Pressure,rad,rad,rad,rad,rad,rad,rad,rad,rad,rad,rad,rad,rad\n'

# Get a line count from the hurdat file
num_lines = 0
with open(hurdat, 'r') as f:
    for line in f:
        num_lines += 1
        
# Clean up the database to make it a little easier to work with
with open(hurdat, 'r') as f:
    with open(hurdat_reformat, 'w') as output:
        output.write(ColHead)
        for i in range(num_lines):
            line = f.readline()
            if len(line) == 38:
                storm = line.split(',')[0]
                storm_name = line.split(',')[1].strip()
            else:
                newline = storm + ',' +  storm_name + ',' + line
                output.write(newline)

### 3. Geoprocessing Step
    See steps in overview for geoprocessing 
    inner_buffer: For the area of interest, select points and give a value of 'y' if in the inner buffered region
    

In [4]:
# Field names created in step 3 for lat, longs in state plane & Area of Interest buffer (inside = 'y')
state_plane_x = 'x_st_pln' 
state_plane_y = 'y_st_pln'
inner_buffer = 'withhin_radius'

    
### 4. Read in Post-Geoprocessed data & format for processing

In [5]:
# Read in data from GIS table
df = pd.read_csv(os.path.join(root_dir, hurdat_post_gis))

# Clean up data, create datetime objects
df['Date'] = df['Date'].astype('str')
df['Year'] = df['Date'].str[0:4]
df['Year'] = pd.to_numeric(df['Year']) + 100
df['Year'] = df['Year'].astype(str)
df['MonthDay'] = df['Date'].str[4:]
df['HourUTC'] = df['HourUTC'].astype('str')
df['HourUTC'].replace(to_replace = '0' , value='0000', inplace=True)
df['HourUTC'].replace(to_replace = '600' , value='0600', inplace=True)
df['dtm'] = pd.to_datetime(df['Year']  + df['MonthDay'] + df['HourUTC'], format = '%Y%m%d%H%M', utc=True )

# Get a list of unique storms 
storms = df['DateID'].unique()

df.head(2)

,OBJECTID,DateID,Name,Date,HourUTC,RecID,Status,Lat,Long,Lat_D,...,rad_12__19,rad_12__20,rad_12__21,rad_12__22,x_st_pln,y_st_pln,withhin_radius,Year,MonthDay,dtm
0,1,AL041851,UNNAMED,18510818,0600,,HU,16.9N,64.1W,16.9,...,-999,-999,-999,,1.471968e+06,322429.31250,n,1951,0818,1951-08-18 06:00:00
1,2,AL041851,UNNAMED,18510818,1200,,HU,17.2N,66.0W,17.2,...,-999,-999,-999,,8.074182e+05,426359.90625,y,1951,0818,1951-08-18 12:00:00


### 5. For each storm, compute forward speed based on hurdat points in area of interest
    *HURDAT data points given every 6 hours. Forward velocities computed using v = x*t where x is the change in space (calculated by applying Pythagorean theorem to x,y from geoprocessing) and t= 6 hours

In [6]:
# Initialize dict to store computed velocites from state plane coordinates
velocity_dict = {}

# conversiton factor
feet_per_hour_to_knots = 0.000164579 #for velocity calculations (fph)

# Calculate Forward velocity (knots) 
# Mean, min and max are calculated.
for storm in storms:
    idxs = df.loc[df['DateID'] == storm].index
    storm_dates = df['dtm'][idxs]
    storm_dates_sorted = storm_dates.sort_values()
    sorted_idx = storm_dates_sorted.index

    xs = []
    ys = []
    speeds=[]
    
    for idx in sorted_idx:
        x = df[state_plane_x].iloc[idx]
        y = df[state_plane_y].iloc[idx]
        xs.append(x)
        ys.append(y)

    for i, x in enumerate(xs):
        if i == 0:
            x0 = xs[i]
            y0 = ys[i]
        else:
            y = ys[i]
            x = xs[i]
            distance = np.sqrt((x-x0)**2 + (y-y0)**2)
            x0 = xs[i]
            y0 = ys[i]
            velocity_fph = distance/6
            forward_speed_knots = velocity_fph*feet_per_hour_to_knots
            speeds.append(forward_speed_knots)
            
    if not len(speeds) == 0:
        mu_v =  np.mean(speeds)  
        min_v =  np.min(speeds) 
        max_v =  np.max(speeds) 
        velocity_dict[storm] =  (mu_v, min_v,max_v )  

df_vs = pd.DataFrame.from_dict(velocity_dict, orient = 'index')
df_vs.rename(columns = {0:'mean_vel', 1:'min_vel', 2:'max_vel'}, inplace=True )
df_vs.head()

,mean_vel,min_vel,max_vel
AL041851,16.988102,15.811507,18.450052
AL011852,8.222860,7.763282,8.682437
AL021852,12.297095,10.011507,16.648908
AL041852,7.203842,6.436254,8.134864
AL041855,14.147129,12.509381,16.763479



### 6. Select data of interest & Merge computed velocities into a new dataframe

In [7]:
# Print columns & select those to include
df.columns

Index(['OBJECTID', 'DateID', 'Name', 'Date', 'HourUTC', 'RecID', 'Status',
       'Lat', 'Long', 'Lat_D', 'Long_D', 'WindSpd', 'Pressure', 'rad', 'rad_1',
       'rad_12', 'rad_12_13', 'rad_12__14', 'rad_12__15', 'rad_12__16',
       'rad_12__17', 'rad_12__18', 'rad_12__19', 'rad_12__20', 'rad_12__21',
       'rad_12__22', 'x_st_pln', 'y_st_pln', 'withhin_radius', 'Year',
       'MonthDay', 'dtm'],
      dtype='object')

In [8]:
# 'Witihin radius' is what we want. Points buffered at a bigger radius to 
# capture multiple points for velocity calcs only. Get rid of the outer buffered points
use_cols = ['DateID', 'Date', 'Name', 'Status', 'WindSpd', 'Pressure', inner_buffer, 'Lat_D']
df_prep = df[use_cols].copy()
df_prep = df_prep[df_prep.withhin_radius != 'n']
df_prep.head()

# Merge Datasets
df_out = pd.merge(df_prep, df_vs, right_index=True, left_on='DateID')
df_out['index'] = df_out.index

# Replace null values of pressure from hurdat (-999) to 9999 in 
# order to select minimum value of low central pressure
null_idx = df_out[df_out.Pressure == -999].index
df_out['Pressure'][null_idx] = 9999

# Get indices of minimum pressure values per unique storm (DateID)
min_cp_rcords = df_out[['DateID', 'Pressure']].copy()
min_cp_rcords.drop_duplicates(inplace=True)
idx = min_cp_rcords.loc[min_cp_rcords.groupby('DateID')['Pressure'].idxmin()].index
df_out = df_out.loc[idx]

# Get rid of records with no listed Pressure, then sort by pressure
df_out = df_out[df_out.Pressure != 9999]
df_out = df_out.sort_values(by = 'Pressure')
df_out.head(3)

,DateID,Date,Name,Status,WindSpd,Pressure,withhin_radius,Lat_D,mean_vel,min_vel,max_vel,index
957,AL091979,19790830,DAVID,HU,150,924,y,16.6,9.829820,7.918438,11.166815,957
1282,AL072010,20100831,EARL,HU,115,931,y,20.2,11.788885,10.938583,12.468001,1282
525,AL041928,19280913,UNNAMED,HU,140,931,y,18.0,11.234057,10.319918,12.519124,525


In [9]:
# Create a Year field, and a unique id field (YearName) for convenience 
use_cols = ['DateID', 'Date' ,'Name','Status','WindSpd','Pressure','mean_vel', 'min_vel', 'max_vel', 'Lat_D']
tmp = df_out[use_cols].copy()
tmp['Year'] = tmp['Date'].str[0:4]
tmp['YearName'] = tmp['Name'] +'_'+  tmp['Year'].astype(str) 
tmp.head(3)

,DateID,Date,Name,Status,WindSpd,Pressure,mean_vel,min_vel,max_vel,Lat_D,Year,YearName
957,AL091979,19790830,DAVID,HU,150,924,9.829820,7.918438,11.166815,16.6,1979,DAVID_1979
1282,AL072010,20100831,EARL,HU,115,931,11.788885,10.938583,12.468001,20.2,2010,EARL_2010
525,AL041928,19280913,UNNAMED,HU,140,931,11.234057,10.319918,12.519124,18.0,1928,UNNAMED_1928


### 7. Code in data from [Reference Guidance](http://www.nws.noaa.gov/oh/hdsc/Technical_memoranda/TM23.pdf) (Location specific) to augment statistical dataset

In [10]:
# NWS Storms (Hand coded from pdf tables)
nws_storm_pressures = [952, 990, 962, 978, 929, 952, 973, 983, 1000, 992, 942, 978, 954, 938, 987, 932, 940]
nws_storm_names = [1947, 1950, 1950, 1951, 1953, 1955, 1956, 1958, 1958, 1960, 1960, 1963, 1963, 1964, 1966, 1966, 1967]
nws_radius_max = [-999, 8, 18, -999, 11 , 12, 14, 20, -999, 8, 13, -999, 10, 6, 15, 4, 12]
nws_velocity = [15, 11, 10, 22, 16, 16, 16, 16, 16, 15, 13, 10, 9, 15, 15, 14, 8]
nws_storm_years = ['UNNAMED', 'BAKER', 'DOG', 'CHARLIE', 'CAROL', 'CONNIE', 'BETSY', 'ELLA', 'FIFI', 'ABBY',
                  'DONNA', 'EDITH', 'FLORA', 'CLEO', 'FAITH', 'INEZ', 'BEULA']

df_nws = pd.DataFrame(list(zip(nws_storm_pressures, nws_storm_names, nws_storm_years)))
df_nws.rename(columns={0:'NWS_Pressure', 1:'Year', 2:'Name'}, inplace=True) 
df_nws['NWS_Year'] = df_nws['Year'].astype(str) 
df_nws['YearName'] = df_nws['Name'] +'_'+  df_nws['Year'].astype(str) 
df_nws['NWS_Rmax'] = nws_radius_max
df_nws['NWS_Velocity'] = nws_velocity
df_nws.drop(axis=1, labels=['Year', 'Name'], inplace=True)
df_nws.head(3)

,NWS_Pressure,NWS_Year,YearName,NWS_Rmax,NWS_Velocity
0,952,1947,UNNAMED_1947,-999,15
1,990,1950,BAKER_1950,8,11
2,962,1950,DOG_1950,18,10


### 8. Merge the datset from the reference case & clean up the completed datset

In [11]:
df_final = pd.merge(tmp, df_nws, left_on='YearName', right_on='YearName', how = 'outer')
df_final.columns

Index(['DateID', 'Date', 'Name', 'Status', 'WindSpd', 'Pressure', 'mean_vel',
       'min_vel', 'max_vel', 'Lat_D', 'Year', 'YearName', 'NWS_Pressure',
       'NWS_Year', 'NWS_Rmax', 'NWS_Velocity'],
      dtype='object')

In [12]:
# Choose the columns to include
use_cols = ['DateID','YearName','Year', 'NWS_Year', 'Status', 'WindSpd', 'Pressure', 
            'NWS_Pressure', 'NWS_Velocity','NWS_Rmax' ,'mean_vel', 'min_vel', 'max_vel', 'Lat_D']

df_final = df_final[use_cols]
df_final['Year'].fillna(0, inplace=True)
fill_idx = df_final[df_final.Year == 0 ].index
df_final['Year'].iloc[fill_idx] = df_final['NWS_Year']
df_final.head(3)

,DateID,YearName,Year,NWS_Year,Status,WindSpd,Pressure,NWS_Pressure,NWS_Velocity,NWS_Rmax,mean_vel,min_vel,max_vel,Lat_D
0,AL091979,DAVID_1979,1979,NaN,HU,150.0,924.0,NaN,NaN,NaN,9.829820,7.918438,11.166815,16.6
1,AL072010,EARL_2010,2010,NaN,HU,115.0,931.0,NaN,NaN,NaN,11.788885,10.938583,12.468001,20.2
2,AL041928,UNNAMED_1928,1928,NaN,HU,140.0,931.0,NaN,NaN,NaN,11.234057,10.319918,12.519124,18.0


### 8. Select storm categories to include (e.g. Hurricanes) & combine Central Pressure values into a single column

In [13]:
# Sort data & Eliminate storms that are not designated as hurricanes
df = df_final.sort_values(by='Year').copy()
drop_idx = df.loc[df['Status'].isin([' TD',' WV', ' TS', ' LO'])].index
df.drop(axis = 0, labels= drop_idx, inplace =True)
df.reset_index(inplace=True)

# Create a new column for the complete central pressure dataset (from hurdat & reference)
df['CP'] = df['Pressure']
fill_idx = df[df.CP == 0.0 ].index
df['CP'].iloc[fill_idx] = df['NWS_Pressure']
df['CP'] = df['CP'].fillna( df['NWS_Pressure'])
df.head(3)

,index,DateID,YearName,Year,NWS_Year,Status,WindSpd,Pressure,NWS_Pressure,NWS_Velocity,NWS_Rmax,mean_vel,min_vel,max_vel,Lat_D,CP
0,11,AL091867,UNNAMED_1867,1867,NaN,HU,100.0,952.0,NaN,NaN,NaN,15.033419,14.279684,16.200408,18.4,952.0
1,13,AL041871,UNNAMED_1871,1871,NaN,HU,100.0,962.0,NaN,NaN,NaN,12.355141,10.670382,14.183307,18.9,962.0
2,23,AL021876,UNNAMED_1876,1876,NaN,HU,70.0,991.0,NaN,NaN,NaN,13.918781,11.436816,15.500666,18.3,991.0


In [14]:
df.columns

Index(['index', 'DateID', 'YearName', 'Year', 'NWS_Year', 'Status', 'WindSpd',
       'Pressure', 'NWS_Pressure', 'NWS_Velocity', 'NWS_Rmax', 'mean_vel',
       'min_vel', 'max_vel', 'Lat_D', 'CP'],
      dtype='object')

In [15]:
# Remove unwanted columns
df.drop(axis=1, labels = ['NWS_Pressure', 'Pressure', 'NWS_Year', 'Status', 'index'], inplace=True)
df.head(3)

,DateID,YearName,Year,WindSpd,NWS_Velocity,NWS_Rmax,mean_vel,min_vel,max_vel,Lat_D,CP
0,AL091867,UNNAMED_1867,1867,100.0,NaN,NaN,15.033419,14.279684,16.200408,18.4,952.0
1,AL041871,UNNAMED_1871,1871,100.0,NaN,NaN,12.355141,10.670382,14.183307,18.9,962.0
2,AL021876,UNNAMED_1876,1876,70.0,NaN,NaN,13.918781,11.436816,15.500666,18.3,991.0


### 9. Calculate Radius to Max Wind for all storms & write the final dataset to file

In [16]:
# Caluclate Rmax & write complete dataset to file
def CalcRmax(vmax_gl, phi):
    #Rmax = 46.4 e^(- 0.0155 vmax_gl) + 0.0169φ
    #https://rdrr.io/cran/stormwindmodel/man/will7a.html
    Rmax = 46.4*np.e**(- 0.0155*vmax_gl) + 0.0169*phi
    return Rmax

df['Calc_Rmax'] = CalcRmax(df['WindSpd'], df['Lat_D'])
df.head()

,DateID,YearName,Year,WindSpd,NWS_Velocity,NWS_Rmax,mean_vel,min_vel,max_vel,Lat_D,CP,Calc_Rmax
0,AL091867,UNNAMED_1867,1867,100.0,NaN,NaN,15.033419,14.279684,16.200408,18.4,952.0,10.159266
1,AL041871,UNNAMED_1871,1871,100.0,NaN,NaN,12.355141,10.670382,14.183307,18.9,962.0,10.167716
2,AL021876,UNNAMED_1876,1876,70.0,NaN,NaN,13.918781,11.436816,15.500666,18.3,991.0,15.987913
3,AL031899,UNNAMED_1899,1899,120.0,NaN,NaN,11.184577,8.158923,14.641040,18.0,940.0,7.527410
4,AL031921,UNNAMED_1921,1921,100.0,NaN,NaN,8.060095,7.037312,9.225871,16.9,961.0,10.133916


In [17]:
df.to_csv(os.path.join(root_dir,processed_data),  sep = '\t')